In [1]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil
from tqdm import tqdm
#import japanize_matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
#from sciplotlib import style as spstyle
import seaborn as sns
from sklearn.model_selection import train_test_split,StratifiedGroupKFold

def search_object(sentence,organ):
    ##構造化文章単位を入力として、特定臓器の情報が入っているかを調べる。
    ##入力 sentence
    ##出力  IO+AEのtoken,IO token,AE token,IOのcertainty_score,IOのtype
    img_obs = sentence['clinical_object']
    mdfs = sentence['modifiers']
    ae_tokens = []

    for i in range(len(mdfs)):
        mdf = mdfs[i]
        if mdf['type'] == 'Anatomical_entity':
            ae_tokens += mdf['tokens']
    
    keywords = ae_tokens+img_obs['tokens']
    #print(keywords)
    if organ in keywords:
        keywords = ''.join(ae_tokens+img_obs['tokens'])
        io_tokens = ''.join(img_obs['tokens'])
        ae_tokens = ''.join(ae_tokens)
        return keywords,io_tokens,ae_tokens,img_obs['certainty_score'],img_obs['type']
    else:
        return None,None,None,None,None
def search_clinicalfinding(modifiers):
    ##clinical_descriptor clinical_discriptorが文中に含まれているかを判別
    ##入力 modifiers
    ##出力  含まれるかどうかbool,clinicalと判断されたwords(なければ0を入れる。)
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if modifier['type'] == 'Clinical_finding':
            return modifier['certainty_score'],''.join(modifier['tokens'])
        else:continue
    return None,'not_found'
def to_multi_label(df,abnormal_list):
    for abnormal_label in abnormal_list:
        df[abnormal_label] = df['label'].apply(lambda x: abnormal_label in x).astype(int)
    df['nofinding'] = df[abnormal_list].sum(axis=1) == 0
    return df

In [2]:
###全体のラベル作成
###observationは絶対に文中にあるという前提。
###

df = pd.read_pickle('../server_data/all_data_20211001-20220605.zip')

organs = ['心','肝','胆','膵','脾','副腎','腎']
os.makedirs('../output',exist_ok=True)
counter = 0
jmid_names = df['jmid_name']
json_lists = df['FINDING_JSON']
#def detect_change(modifiers):
change_word_list = []
clinical_word_list = []
data = []
counter = 0
for jmid_name,json_load in zip(jmid_names,json_lists):
    for organ in organs:
        #print(organ)
        for i in range(len(json_load)):
            all_keyword,io_keyword,ae_keyword,certainty,img_type = search_object(json_load[i],organ)
            clinical_scale,clinical_word = search_clinicalfinding(json_load[i]['modifiers'])

            
            if all_keyword is not None:
                counter+=1
                data.append([jmid_name,organ,all_keyword,io_keyword,ae_keyword,certainty,img_type,clinical_word,clinical_scale])
            else:
                continue

data = pd.DataFrame(data,columns=['jmid_name','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','obs_name','clin_findings','clin_certainty'])
data['jmid_name'] = data['jmid_name'].apply(lambda x:x.split('/')[-1])

In [3]:
data_spleen = data[(data['obs_name'] == 'Imaging_observation') & (data['organ'] == '脾')].reset_index(drop=True)

In [7]:
print([i for i in data_spleen[data_spleen['obs_certainty'] == 0]['io_tokens'].value_counts().index])

['異常', '所見', '異常所見', '脾腫', '有意な異常', '肝脾腫', 'リンパ節腫大', '病変', '粗大病変', '腫大', '事項', '腹水', '血栓', '腫大リンパ節', '口径不整', '高度肝脾腫', '狭窄', '脾腫大', 'SOL', '造影不領域', '脾腫瘤', '副血行路発達像', '閉塞', '仮性動脈瘤', '骨破壊像', '前立腺腫大', '腫瘤', '水腎', '粗大腫瘤', '脾損傷', '広狭不整', '尿管結石', '腫瘤性病変', '処置', '瘤', '嚢状瘤', 'LDA', '事項（-）', '結節', '主膵管拡張', '胆管拡張', '異常壁肥厚像', '新出病変', '骨病変肝脾腫', '肝脾腫大', '腹水貯留', '血腫', '閉塞機転', '造影剤の流入', '異常病変', '脂肪織濃度上昇', '造影増強効果', '像', '異常ない異常', '壁肥厚', '濃度上昇', '低吸収域', '腫瘤影']


In [8]:
normal_obs_finding_list = ['異常', '所見', '異常所見', '脾腫', '有意な異常', '肝脾腫', 'リンパ節腫大',
                            '病変', '粗大病変', '腫大', '事項', '腹水', '血栓', '腫大リンパ節', '口径不整', 
                            '高度肝脾腫', '狭窄', '脾腫大', 'SOL', '造影不領域', '脾腫瘤', '副血行路発達像', 
                            '閉塞', '仮性動脈瘤', '骨破壊像', '前立腺腫大', '腫瘤', '水腎', '粗大腫瘤', '脾損傷', 
                            '広狭不整', '尿管結石', '腫瘤性病変', '処置', '瘤', '嚢状瘤', 'LDA', '事項（-）', '結節', 
                            '主膵管拡張', '胆管拡張', '異常壁肥厚像', '新出病変', '骨病変肝脾腫', '肝脾腫大', '腹水貯留', 
                            '血腫', '閉塞機転', '造影剤の流入', '異常病変', '脂肪織濃度上昇', '造影増強効果', '像', '異常ない異常', 
                            '壁肥厚', '濃度上昇', '低吸収域', '腫瘤影']

In [9]:
print([i for i in data_spleen[data_spleen['obs_certainty'] != 0]['io_tokens'].value_counts().index])

['脾腫', '副脾', '腎嚢胞', '腫大', '脾動脈瘤', 'LDA', '腫瘤', 'シャント', '嚢胞', '閉塞', '小嚢胞', '拡張', '石灰化', '低吸収域', '胆石', '狭窄', '血栓', '動脈瘤', '腹水', '嚢胞性病変', '結石', '腎盂嚢胞', '変形', '結節', '軟部影', '脾腫大', '発達', '狭小化', '異常', '脾嚢胞', 'SOL', '壁肥厚', '萎縮', '小結節', '腫瘤影', '腎のう胞', '子宮筋腫', '脾腎シャント', '副血行路発達像', '腎萎縮', 'シャント発達', 'リンパ節腫大', '蛇行', '脾結節', '腎結石', '肥厚', '瘤状拡張', '肝脾腫', '造影不領域', '脾LDA', '瘤', '液貯留', '結腸憩室', '脾腫瘤', '血管腫', '小結石', '脾嚢胞性病変', '石灰化脾動脈瘤', '腫瘤内部', '構造', '脾腫瘍', '小石灰化', '傍腎盂嚢胞', 'くびれ', '血栓化', 'air', 'encasement', '収縮状態', '造影効果', '血腫瘤', '起始しています', '脾石灰化', '造影不良域', '低濃度域', '液体貯留', '水腎症', '小腎嚢胞', '造影不良', '低吸収結節', '血流', '口径', '腎シャント', '緊満', '脾腫同様', '腹膜肥厚', '気腫', '小LDA', '嚢胞性腫瘤', '腫瘍', '血流低下', 'リンパ節', '馬蹄腎', '腎嚢胞retroaortic_left_renal_vein', '梗塞', '胃壁造影効果', '多脾症', '微小嚢胞', '門脈瘤', '腹部大動脈瘤', '小低吸収域', '脂肪織の介在', '側副路発達', '腎短絡', '結節性病変', '造影欠損域', '増生', '嚢状動脈瘤', '部分梗塞', '短絡', '静脈瘤', '解離', '接触', '食道裂孔ヘルニア', '造影不良SOL', 'シャントの発達', '索状LDA', '瘤形成', '分岐', '索状影', '側副路', '血腫', '血栓形成', 'cavernous_transformation', '脾動脈拡張', '圧排変形', '門

**abnormal_obs_finding_list**→異常なラベルとして使用するもの  
**abnormal_obs_finding_id**→異常の大まかなカテゴリ付け。  
**abnormal_obs_finding_renamed**→大まかなカテゴリの名前

In [10]:
abnormal_obs_finding_list = ['脾腫', '腫大', 'LDA', '腫瘤', '嚢胞', '小嚢胞', '石灰化', '低吸収域', '嚢胞性病変', 
                            '結石', '変形', '結節', '脾腫大', '軟部影', '脾嚢胞', 'SOL', 
                            '萎縮', '壁肥厚', '小結節', '腫瘤影', '脾結節', '肥厚', '肝脾腫', '造影不領域', '脾LDA', 
                            '液貯留',  '脾腫瘤', '小結石', '血管腫', '脾嚢胞性病変','腫瘤内部', '脾腫瘍', '小石灰化']
###normal findingsの所見idは0として、abnobal findingsの所見idをカテゴリごとに割り振る(i.e. 嚢胞→１、石灰化→2,脂肪肝→3などなど)
### マルチラベルにする。
##正常として使うラベルを指定する。
#normal_obs_finding_list = ['SOL', '異常', '所見', '病変', '腫瘤影', '腫瘤', '腫瘤性病変','異常所見','占拠性病変', '有意な異常','新出病変','washout']
##異常として使うラベルを指定する。
#abnormal_obs_finding_list = ['嚢胞', '拡張', '小嚢胞', '腫瘤', 'LDA', '血管腫', '脂肪肝', '石灰化','肝嚢胞', '早期濃染像', 'pneumobilia']
##疾患のグループにまとめる。
abnormal_obs_finding_id = [0,1,4,4,2,2,3,4,2,3,1,4,1,4,2,4,1,4,4,4,4,4,4,4,4,4,4,3,4,2,4,4,3]

abnormal_obs_finding_list_renamed = ['脾腫', '形態変化','嚢胞','石灰化', 'other_abnormality']
print(len(abnormal_obs_finding_list),len(abnormal_obs_finding_id))

33 33


**セグメンテーションが使えるもの　かつ　異常(正常)ラベルが定義されているもの**を学習に使用  
データは検査ごとにまとめる。


In [11]:
use_normalfindings = (data_spleen['io_tokens'].apply(lambda x:x in normal_obs_finding_list)) & (data_spleen['obs_certainty'] == 0)
use_abnormalfindings = (data_spleen['io_tokens'].apply(lambda x:x in abnormal_obs_finding_list)) & (data_spleen['obs_certainty'] != 0)
available_segmentation_list = glob.glob('../data/spleen_seg_img/*.nii.gz') 
available_segmentation_list = [os.path.basename(p) for p in available_segmentation_list]
use_segmentation = data_spleen['jmid_name'].apply(lambda x:x in available_segmentation_list)
# use_abnormalfindings = (data_spleen['clin_findings'].str.contains('癌') | data_spleen['clin_findings'].str.contains('IPMN'))
# use_certainty = ((data_spleen['obs_certainty'] == 0) | (data_spleen['obs_certainty'] == 4) )
data_spleen['use_training'] = (use_normalfindings|use_abnormalfindings) & use_segmentation 
data_spleen['label'] = 'not used'
data_spleen.loc[use_abnormalfindings,'label'] = data_spleen.loc[use_abnormalfindings,'io_tokens'].apply(lambda x:abnormal_obs_finding_list_renamed[abnormal_obs_finding_id[abnormal_obs_finding_list.index(x)]])
data_spleen.loc[use_normalfindings,'label'] = 'nofinding'
tmp = data_spleen.groupby('jmid_name')['label'].apply(lambda x: x.tolist())
data_spleen.drop('label',axis=1,inplace=True)
data_spleen = data_spleen.merge(tmp,how='left',on='jmid_name')


In [12]:
data_spleen = to_multi_label(data_spleen,abnormal_obs_finding_list_renamed)

data_spleen = data_spleen.drop_duplicates(subset='jmid_name')
print(data_spleen.shape)
data_spleen_use = data_spleen[data_spleen['use_training'] == True].reset_index(drop=True)
data_spleen_use['file'] = data_spleen_use['jmid_name']
data_spleen_use['jmid_name'] = data_spleen_use['jmid_name'].apply(lambda x:'_'.join(x.split('_')[:-1])+'_0000.nii.gz')

print(data_spleen_use.shape)
##元の所見文と結合
data_spleen_use = data_spleen_use.merge(df,on='jmid_name',how='left').drop_duplicates(subset='jmid_name')
print(data_spleen_use.shape)
data_spleen_use[abnormal_obs_finding_list_renamed+['nofinding']].sum(axis=0)

(16031, 17)
(12378, 18)
(12378, 29)


脾腫                   2063
形態変化                  579
嚢胞                    291
石灰化                   127
other_abnormality     652
nofinding            8783
dtype: int64

In [ ]:
## cleanなラベルは検証データとして使用する。学習データはそれ以外を使用。
clean_df = pd.read_csv('../data/spleen_dataset_test_clean.csv')
jmid_files_clean = clean_df['file'].values
tmp = data_spleen_use[data_spleen_use['jmid_name'].apply(lambda x:x in jmid_files_clean)]
test_groups = tmp['FACILITY_CODE'].astype(str)+tmp['ACCESSION_NUMBER'].astype(str)
groups = data_spleen_use['FACILITY_CODE'].astype(str)+data_spleen_use['ACCESSION_NUMBER'].astype(str)


df_train = data_spleen_use[groups.apply(lambda x:x not in test_groups.tolist())]
df_test =  data_spleen_use[groups.apply(lambda x:x in test_groups.tolist())]


columns = ['file','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','FINDING','FINDING_JSON','FACILITY_CODE','ACCESSION_NUMBER']+abnormal_obs_finding_list_renamed+['nofinding']
df_train[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/liver_dataset_train_multi5.csv',index=False)
df_test[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/liver_dataset_test_multi5.csv',index=False)